# # User Model Definition

In [1]:
# The `NubisonModel` class serves as a base class for creating custom user model classes.
from nubison_model import NubisonModel

class UserModel(NubisonModel):
    """
    A user model that extends the `NubisonModel` base class.
    """
    def load_model(self) -> None:
        """
        This method is used to load the model weights from the file.
        """
        try:
            # Import the SimpleLinearModel class from the src directory
            # This class implements a basic linear model for text processing
            # Using absolute import path to ensure reliable imports across different execution contexts
            from src.SimpleLinearModel import SimpleLinearModel
            self.model = SimpleLinearModel("./src/weights.txt")
        except Exception as e:
            print(f"Error: {e}")

    def infer(self, x1: float, x2: float):
        """
        This method is used to return the inference result.
        """
        return {"y": self.model.calc(x1, x2)}

# # Model Register

In [5]:
# The `register` function is utilized to register the user-defined model with the system,
from nubison_model import register

# Register the user model
# The `artifact_dirs` argument specifies the folders containing the files used by the model class.
model_id = register(UserModel(), mlflow_uri="https://model.nubison.io", model_name="nubison_model", artifact_dirs="src", params={"desc": "This is a test model"}, metrics={"train": 0.9, "validation": 0.8, "test": 0.7})
print(f"Model registered: {model_id}")


2024/11/28 16:58:19 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - nubison-model (current: 0.0.2.dev3+3e1558a.20241118053748, required: nubison-model==0.0.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/11/28 16:58:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'nubison_model' already exists. Creating a new version of this model...
2024/11/28 16:58:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nubison_model, version 8
Created version '8' of model 'nubison_model'.
2024/11/28 16:58:20 INFO mlflow.tracking._tracking_service.client: 🏃 

Model registered: runs:/dbebfd0f99594a0fa1c67a6a00b3e270/


# # Model Testing

In [6]:
# The `test_client` function is used to test the model.
from nubison_model.Service import test_client

# Create a test client for the model.
with test_client(model_id) as client:
    # The request should include the parameters defined in the `infer` method.
    result = client.post("/infer", json={"x1": 3.1, "x2": 2})
    print(f"The result of the linear model is {result.json()['y']}.")



2024/11/28 14:29:26 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - nubison-model (current: 0.0.2.dev3+3e1558a.20241118053748, required: nubison-model==0.0.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


2024/11/28 14:29:26 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - nubison-model (current: 0.0.2.dev3+3e1558a.20241118053748, required: nubison-model==0.0.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024-11-28 14:29:26,359 - SimpleLinearModel - INFO - Weights loaded successfully from ./src/weights.txt.
INFO:SimpleLinearModel:Weights loaded successfully from ./src/weights.txt.
2024-11-28 14:29:26,367 - SimpleLinearModel - INFO - Calculating the result of the linear model with x1=3.1, x2=2.0.
INFO:SimpleLinearModel:Calculating the result of the linear model with x1=3.1, x2=2.0.


Prepared artifact: src -> /tmp/tmprsct2rwt/artifacts/src
The result of the linear model is 4.35.
